In [1]:
#pip install ip2geotools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
inputfilename_train = "dataset/rba_train.csv"
inputfilename_test = "dataset/rba_test.csv"

strat_train_set = pd.read_csv(inputfilename_train)
strat_test_set = pd.read_csv(inputfilename_test)

In [3]:
strat_train_set.columns

Index(['Unnamed: 0', 'index', 'Login Timestamp', 'User ID',
       'Round-Trip Time [ms]', 'IP Address', 'Country', 'Region', 'City',
       'ASN', 'User Agent String', 'Browser Name and Version',
       'OS Name and Version', 'Device Type', 'Login Successful',
       'Is Attack IP', 'Is Account Takeover', 'year', 'month', 'day', 'hour',
       'minute', 'second', 'millisecond', 'Is Failed Login', 'Login counter',
       'Is Attack'],
      dtype='object')

##### Modification 1: Drop unnecessary colums
'Unnamed: 0', 'index','Login Timestamp' 'Country', 'Region', 'City','Browser Name and Version',
       'OS Name and Version', 'Device Type''Is Attack IP', 'Is Account Takeover','second', 'millisecond''Login counter'

In [4]:
strat_train_set.drop(['Unnamed: 0', 'index','Login Timestamp', 'Country', 'Region', 'City','Browser Name and Version',\
       'OS Name and Version','Is Attack IP', 'Is Account Takeover','second', 'millisecond','Login counter'],  axis='columns', inplace=True)

strat_test_set.drop(['Unnamed: 0', 'index','Login Timestamp', 'Country', 'Region', 'City','Browser Name and Version',\
       'OS Name and Version', 'Is Attack IP', 'Is Account Takeover','second', 'millisecond','Login counter'],  axis='columns', inplace=True)
  

In [5]:
strat_test_set.columns

Index(['User ID', 'Round-Trip Time [ms]', 'IP Address', 'ASN',
       'User Agent String', 'Device Type', 'Login Successful', 'year', 'month',
       'day', 'hour', 'minute', 'Is Failed Login', 'Is Attack'],
      dtype='object')

##### Modification 2: Drop the rows where RTT is null

In [6]:
strat_train_set.dropna(subset=["Round-Trip Time [ms]"],inplace=True)
strat_test_set.dropna(subset=["Round-Trip Time [ms]"],inplace=True)

In [7]:
strat_train_set['hour'].value_counts()

18    908
16    891
19    887
17    867
14    801
15    790
10    763
9     757
20    756
13    735
12    731
11    689
8     670
21    546
7     487
6     354
22    337
5     204
23    161
4      98
0      94
1      49
3      46
2      45
Name: hour, dtype: int64

##### Modification 3:  Create a new column for storing time of login (3 possible values: Early Morning, Day, Late Night)
'Early Morning' if Hour = 3 to 7
'Late Night' if hour = 22,23,0,1,2
'Day' otherwise 8 to 21

In [8]:
def finddayslot(hour):
    slotOptions =['early morning', 'day', 'late night']
    if hour in [3,4,5,6,7]:
        return slotOptions[0]
    elif hour in [22,23,0,1,2]:
        return slotOptions[2]
    else: 
        return slotOptions[1]
    
strat_train_set['Day Slot'] = strat_train_set['hour'].map(finddayslot)
strat_test_set['Day Slot'] = strat_test_set['hour'].map(finddayslot)
display(strat_train_set.head())

,User ID,Round-Trip Time [ms],IP Address,ASN,User Agent String,Device Type,Login Successful,year,month,day,hour,minute,Is Failed Login,Is Attack,Day Slot
0,-4324475583306591935,506.0,80.232.46.22,25400,Mozilla/5.0 (iPhone; CPU iPhone OS 10_1_1 lik...,mobile,0,2021,2,20,9,58,1,0,day
40,4141948673144054545,1128.0,84.208.181.38,41164,Mozilla/5.0 (iPhone; CPU iPhone OS 12_0_1 lik...,mobile,1,2020,3,17,20,5,0,0,day
54,-8152822881938774781,551.0,84.209.190.114,41164,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,mobile,1,2020,6,28,11,59,0,0,day
59,1412256765887180107,506.0,79.161.83.66,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,mobile,0,2020,11,3,23,54,1,0,late night
64,2843739543340219881,43.0,10.0.160.76,500136,Mozilla/5.0 (Linux; Android 6.9; SM-J500H/DS ...,mobile,1,2020,8,4,9,54,0,0,day


####Modification 4: Remove rows that have private IP in IP address.
Private Address Ranges
Class A: 10.0. 0.0 to 10.255. 255.255.
Class B: 172.16. 0.0 to 172.31. 255.255.
Class C: 192.168. 0.0 to 192.168. 255.255.

In [9]:
'''# delete all rows with IP address as private IP

indexes_1= strat_train_set[strat_train_set['IP Address'].str.startswith("10.")].index
indexes_2= strat_train_set[strat_train_set['IP Address'].str.startswith("172.")].index
indexes_3= strat_train_set[strat_train_set['IP Address'].str.startswith("192.168.")].index

print(len(indexes_1))
print(len(indexes_2))
print(len(indexes_3))
strat_train_set.drop(indexes_1 , inplace=True)
strat_train_set.drop(indexes_2 , inplace=True)
strat_train_set.drop(indexes_3 , inplace=True)
#strat_train_set.head(15)'''

'# delete all rows with IP address as private IP\n\nindexes_1= strat_train_set[strat_train_set[\'IP Address\'].str.startswith("10.")].index\nindexes_2= strat_train_set[strat_train_set[\'IP Address\'].str.startswith("172.")].index\nindexes_3= strat_train_set[strat_train_set[\'IP Address\'].str.startswith("192.168.")].index\n\nprint(len(indexes_1))\nprint(len(indexes_2))\nprint(len(indexes_3))\nstrat_train_set.drop(indexes_1 , inplace=True)\nstrat_train_set.drop(indexes_2 , inplace=True)\nstrat_train_set.drop(indexes_3 , inplace=True)\n#strat_train_set.head(15)'

###Modification 4: Create a new columns for storing lattitude, longitude (ASN is already in data), based on IP
longitude*1000+latitude

In [10]:
'''import geocoder

strat_train_set['coordinate'] = strat_train_set['IP Address'].apply(lambda x: geocoder.ip(x).lng*1000+geocoder.ip(x).lat)
strat_train_set.head()'''


"import geocoder\n\nstrat_train_set['coordinate'] = strat_train_set['IP Address'].apply(lambda x: geocoder.ip(x).lng*1000+geocoder.ip(x).lat)\nstrat_train_set.head()"

#### Modification 5: Create 3 new columns for storing browser family, OS family, is Bot, based on User Agent

In [11]:
from user_agents import parse
def findBrowserFamily(ua_string):
    user_agent = parse(ua_string)
    return user_agent.browser.family
def findOSFamily(ua_string):
    user_agent = parse(ua_string)
    return user_agent.os.family
def isBot(ua_string):
    user_agent = parse(ua_string)
    return user_agent.is_bot

    
strat_train_set['Browser Family'] = strat_train_set['User Agent String'].map(findBrowserFamily)
strat_train_set['OS Family'] = strat_train_set['User Agent String'].map(findOSFamily)
strat_train_set['Is Bot'] = strat_train_set['User Agent String'].map(isBot)

strat_test_set['Browser Family'] = strat_test_set['User Agent String'].map(findBrowserFamily)
strat_test_set['OS Family'] = strat_test_set['User Agent String'].map(findOSFamily)
strat_test_set['Is Bot'] = strat_test_set['User Agent String'].map(isBot)


strat_train_set.head()

,User ID,Round-Trip Time [ms],IP Address,ASN,User Agent String,Device Type,Login Successful,year,month,day,hour,minute,Is Failed Login,Is Attack,Day Slot,Browser Family,OS Family,Is Bot
0,-4324475583306591935,506.0,80.232.46.22,25400,Mozilla/5.0 (iPhone; CPU iPhone OS 10_1_1 lik...,mobile,0,2021,2,20,9,58,1,0,day,Mobile Safari,iOS,False
40,4141948673144054545,1128.0,84.208.181.38,41164,Mozilla/5.0 (iPhone; CPU iPhone OS 12_0_1 lik...,mobile,1,2020,3,17,20,5,0,0,day,Mobile Safari,iOS,False
54,-8152822881938774781,551.0,84.209.190.114,41164,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,mobile,1,2020,6,28,11,59,0,0,day,Mobile Safari UI/WKWebView,iOS,False
59,1412256765887180107,506.0,79.161.83.66,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,mobile,0,2020,11,3,23,54,1,0,late night,Chrome,iOS,False
64,2843739543340219881,43.0,10.0.160.76,500136,Mozilla/5.0 (Linux; Android 6.9; SM-J500H/DS ...,mobile,1,2020,8,4,9,54,0,0,day,Android,Android,False


In [12]:
strat_train_set.to_csv("dataset/rba_train_clean.csv",index=False)
strat_test_set.to_csv("dataset/rba_test_clean.csv",index=False)